### Prepare raw data

Download html files from site

In [1]:
#!wget -r -k -l 7 -p -E -nc https://povar.ru/ratingCountratingCount
!rm -rf ./chroma_db

In [2]:
# !apt-get update
# !apt-get install unzip
# !unzip povar_recipes_1.zip
# !unzip povar_recipes_2.zip
# !unzip povar_recipes_3.zip

### Make DataSet

In [3]:
%load_ext autoreload
%autoreload
from utils import make_data_set, read_recipes_pkl, save_recipes_pkl, make_recipes,filter_recipe

%load_ext autoreload
%autoreload
from Recipe import Recipe, RecipesProject

MAIN_DIR = "/home/zaderu/Documents/MLHS/LLM Project/data/povar.ru/recipes/"
FILE_NAME = "/workspace/recipe_project/llm_project/povar_recipes_"

/workspace/recipe_project/llm_venv/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# povar_recipes = make_data_set(MAIN_DIR)
# save_recipes_pkl(povar_recipes, FILE_NAME)

povar_recipes = []
for i in range(1, 4):
    _povar_recipes = read_recipes_pkl(FILE_NAME + str(i) + ".pickle")
    povar_recipes.extend(_povar_recipes)
len(povar_recipes)

Collected 22000 recipes
Collected 22000 recipes
Collected 22349 recipes


66349

In [14]:
recipes_list = make_recipes(povar_recipes)

Total 66349 recipes is processed.


In [15]:
recipes_list = filter_recipe(recipes_list, max_steps = 10, max_min=120, min_rating=4, min_votes=2)
save_recipes_pkl(recipes_list, "./Recipe_final.pickle")

32330 recipes are selected.


In [16]:
recipes_list[0].document.page_content

'Сосиски в лаваше жареные лаваш тонкий, сосиски, сыр твердый, соусЛаваш расстилаем на доске. Чтобы не крошился - можно немного его подержать над паром, станет пластичнее. Смазываем его соусом на ваше усмотрение, посыпаем обильно тертым сыром., На краешек выкладываем сосиску, сворачиваем рулет. У меня было четыре маленьких сосиски, - с разных сторон лаваша начинаем заворачивать, а посередине обрезаем., Обжариваем на масле или на сухой сковороде со всех сторон., Все готово! Приятного аппетита!'

### Create Chroma DB

In [17]:
# Chroma DB
%load_ext autoreload
%autoreload
from build_chroma_db import build_chroma_db, requst_chroma_db
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
vector_store, tokenizer = build_chroma_db(
    recipes_list,
    embedding_model="ai-forever/sbert_large_nlu_ru", #"nomic-ai/nomic-embed-text-v1.5", #"IlyaGusev/saiga_yandexgpt_8b" 
    device=device,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/workspace/recipe_project/llm_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


39813 chunks are to be loaded to chroma db


### Create Knowledgr Graph

In [18]:
### Knowledge graph
%load_ext autoreload
%autoreload
from build_knowledge_graph import make_tags_list, lemmatize_tags, build_knowledge_graph\
    , lemmatize, lemmatize_sentance, make_one_word_tags_list, enreach_query_with_relative_tags

tags = make_tags_list(recipes_list)

lemmatize_tags(tags)

one_word_tags = make_one_word_tags_list(tags)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
Graph = build_knowledge_graph(recipes_list, tags)

In [20]:
rp = RecipesProject(
    recipes=recipes_list,
    knowledgeGraph=Graph,
    tags=tags,
    vectorStore=vector_store,
    oneWordTags=one_word_tags
)
# rp.add_resipes_list(recipes_list)
# rp.add_knowledge_graph(G)
# rp.add_tags(tags)
# rp.add_one_word_tags(one_word_tags)

### Create LLM

In [22]:
%load_ext autoreload
%autoreload
from build_ollama import build_model, llm_invoke
llm = build_model()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Testing

In [23]:
requests = [
    ["Какой рецепт вегетарианской пасты?"],
    ["Как приготовить блинчики?"],
    ["Что можно приготовить из куриного филе?"],
    ["Как сделать шоколадный торт?"],
    ["Как приготовить рис для суши?"],
    ["Как приготовить суп-пюре из тыквы?"],
    ["Что можно приготовить из картошки и сыра?"],
    ["Как сделать домашний хлеб?"],
    ["Какие десерты можно приготовить без муки?"],
    ["Как приготовить рис с курицей и овощами?"],
    ["Как сделать соус для пасты?"],
    ["Как приготовить морковные котлеты?"],
    ["Что приготовить на ужин за 30 минут?"],
    ["Как сделать домашнюю пиццу?"],
    ["Что можно приготовить из куриного филе, риса и брокколи?"],
    ["Как приготовить сладкий омлет?"],
    ["Как сделать домашнее мороженое без мороженицы?"],
    ["Что можно приготовить для обеда из мяса и риса?"],
    ["Как приготовить крем-суп из грибов?"],
    ["Как приготовить рыбу с картошкой в духовке?"],
    ["Что приготовить для завтрака, если есть яйца, авокадо и помидоры?"],
    ["Как приготовить куриные крылышки на гриле?"],
    ["Что приготовить на праздничный ужин?"],
    ["Как приготовить соус бешамель?"],
    ["Как приготовить кальмары с овощами ?"],
]

### Queries

In [24]:
recipes_dict = {r.id: r for r in rp.recipes}

In [26]:
import random
def final_query(query, n=3):
    query_clean = query.lower().replace("рецепт","").replace("кухня","")
    results_kg = rp.invoke(query_clean, verbose=False)
    if len(results_kg[0]) == 0:
        results_kg = results_kg[1]
    else:
        results_kg = results_kg[0]
    id_kg = [id for id in results_kg]

    results_db = requst_chroma_db(vector_store, tokenizer, query, device, k=100)
    id_db = [r.metadata['id'] for r in results_db]
    
    res_final = []
    for id in results_db:
        if id in id_kg:
            res_final.append(id)
    if len(res_final) < n:
        kg_rest = [id for id in results_kg if id not in id_db]
        res_final.extend(random.sample(kg_rest, k=min([n-len(res_final), len(kg_rest)])))
    if len(res_final) < n:
        db_rest = [id for id in id_db if id not in res_final]
        res_final.extend(db_rest[0:n-len(res_final)])
        
    return id_kg[:n], id_db[:n], res_final[:n]

In [27]:
%load_ext autoreload
%autoreload
from prompt import create_chat_history, prompt, formatted_chat_history, formatted_rag
chat_history = create_chat_history()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import time
from tqdm import tqdm

log = []

for query in tqdm(requests[:10]):
    log.append([])
    chat_history = create_chat_history()
    for question in query:
        start_time = time.time()
        kg, db, final = final_query(question)
        format_chat_history = formatted_chat_history(chat_history)
        format_rag = formatted_rag([recipes_dict[r].document.page_content for r in final])
        question_input = {
            "chat_history": format_chat_history,
            "user_input": question,
            "retrieved_recipes": format_rag
        }
        final_prompt = prompt.format(**question_input)
        llm_prompt_result = llm_invoke(llm, final_prompt)
        llm_prompt_time = time.time() - start_time
        start_time = time.time()
        llm_question_result = llm_invoke(llm, question)
        llm_question_time = time.time() - start_time
        chat_history.add_user_message(question)
        chat_history.add_ai_message(llm_prompt_result)
        log[-1].append({
            'kg':kg,
            'db':db,
            'llm_question':llm_question_result,
            'time_question':llm_question_time,
            'llm_prompt':llm_prompt_result,
            'time_prompt':llm_prompt_time,
        })

  0%|          | 0/10 [00:00<?, ?it/s]


Вы — опытный кулинарный помощник. Ваша цель — давать полезные, креативные и простые в использовании советы по приготовлению пищи.
У вас есть доступ к большой базе рецептов, хранящейся в ChromaDB, которая позволяет вам извлекать соответствующие рецепты и предлагать
персонализированные идеи приготовления пищи на основе предпочтений пользователя, доступных ингредиентов, диетических ограничений и стиля приготовления.

Используйте историю чата, чтобы сохранять контекст и избегать повторений. Если пользователь упомянул определенные ингредиенты,
предпочтения или антипатии ранее в разговоре, учтите это при предоставлении совета.

Используйте выбранные рецепты из ChromaDB в качестве руководства для создания индивидуальных предложений. Вы можете комбинировать элементы из разных рецептов или адаптировать их на основе предпочтений пользователя.

Четко форматируйте ответы, включая:
1. **Название**: название рецепта или предложения.
2. **Ингредиенты**: перечислите ингредиенты с количеством.
3. **Ин

 10%|█         | 1/10 [00:09<01:26,  9.64s/it]


Вы — опытный кулинарный помощник. Ваша цель — давать полезные, креативные и простые в использовании советы по приготовлению пищи.
У вас есть доступ к большой базе рецептов, хранящейся в ChromaDB, которая позволяет вам извлекать соответствующие рецепты и предлагать
персонализированные идеи приготовления пищи на основе предпочтений пользователя, доступных ингредиентов, диетических ограничений и стиля приготовления.

Используйте историю чата, чтобы сохранять контекст и избегать повторений. Если пользователь упомянул определенные ингредиенты,
предпочтения или антипатии ранее в разговоре, учтите это при предоставлении совета.

Используйте выбранные рецепты из ChromaDB в качестве руководства для создания индивидуальных предложений. Вы можете комбинировать элементы из разных рецептов или адаптировать их на основе предпочтений пользователя.

Четко форматируйте ответы, включая:
1. **Название**: название рецепта или предложения.
2. **Ингредиенты**: перечислите ингредиенты с количеством.
3. **Ин

 20%|██        | 2/10 [00:23<01:35, 11.95s/it]

...

 80%|████████  | 8/10 [01:18<00:18,  9.01s/it]


Вы — опытный кулинарный помощник. Ваша цель — давать полезные, креативные и простые в использовании советы по приготовлению пищи.
У вас есть доступ к большой базе рецептов, хранящейся в ChromaDB, которая позволяет вам извлекать соответствующие рецепты и предлагать
персонализированные идеи приготовления пищи на основе предпочтений пользователя, доступных ингредиентов, диетических ограничений и стиля приготовления.

Используйте историю чата, чтобы сохранять контекст и избегать повторений. Если пользователь упомянул определенные ингредиенты,
предпочтения или антипатии ранее в разговоре, учтите это при предоставлении совета.

Используйте выбранные рецепты из ChromaDB в качестве руководства для создания индивидуальных предложений. Вы можете комбинировать элементы из разных рецептов или адаптировать их на основе предпочтений пользователя.

Четко форматируйте ответы, включая:
1. **Название**: название рецепта или предложения.
2. **Ингредиенты**: перечислите ингредиенты с количеством.
3. **Ин

 90%|█████████ | 9/10 [01:26<00:08,  8.58s/it]


Вы — опытный кулинарный помощник. Ваша цель — давать полезные, креативные и простые в использовании советы по приготовлению пищи.
У вас есть доступ к большой базе рецептов, хранящейся в ChromaDB, которая позволяет вам извлекать соответствующие рецепты и предлагать
персонализированные идеи приготовления пищи на основе предпочтений пользователя, доступных ингредиентов, диетических ограничений и стиля приготовления.

Используйте историю чата, чтобы сохранять контекст и избегать повторений. Если пользователь упомянул определенные ингредиенты,
предпочтения или антипатии ранее в разговоре, учтите это при предоставлении совета.

Используйте выбранные рецепты из ChromaDB в качестве руководства для создания индивидуальных предложений. Вы можете комбинировать элементы из разных рецептов или адаптировать их на основе предпочтений пользователя.

Четко форматируйте ответы, включая:
1. **Название**: название рецепта или предложения.
2. **Ингредиенты**: перечислите ингредиенты с количеством.
3. **Ин

100%|██████████| 10/10 [01:34<00:00,  9.44s/it]


In [30]:
for query, ll in zip(requests, log):
    for q, l in zip(query, ll):
        kg = recipes_dict[l['kg'][0]].display
        db = recipes_dict[l['db'][0]].display
        
        print(f"\n\n**Question**: {q}\n\n\t**KG**: {kg}\n\n\t**DB**: {db}\n\n\t**LLM_Prompt**: {l['llm_prompt']}\n\n\t**LLM_Query**: {l['llm_question']}")
    



**Question**: Какой рецепт вегетарианской пасты?

	**KG**: Ужин без мяса

Количество порций: 4.0.
Ингридиенты
	паста: 500 грамм.
	лук: 1 штука.
	чеснок: 2 зубчика.
	сладкий перец: 1 штука.
	грибы шампиньоны: 200 грамм.
	помидоры: 2 штуки.
	сахар: 2 ст. ложки.
	соль, перец: по вкусу.
	сухие итальянские травы: по вкусу.

Способ приготовления
	1. На разогретой сковороде с маслом жарьте измельченный лук и чеснок, пока лук не станет прозрачным. А в отдельной кастрюле отварите макароны до полуготовности..
	2. К луку отправьте нарезанный кубиком перец и грибы. Все перемешайте и тушите 5 минут. После этого добавьте порезанные помидоры, сахар, приправу, соль и перец. Туда же, в сковородку, отправьте пасту и хорошо перемешайте все ингредиенты. Уменьшите огонь, накройте крышкой и тушите 30 минут..
	3. Подавайте пасту порционно. Сверху посыпьте свежей рубленой зеленью..
Калории: 97 ккал.
Белки: 3 г.
Жиры: 2 г.
Углеводы: 14 г.
Время: 50 мин.


	**DB**: Ленивая ачма

Количество порций: 10.0.
Ингри

In [31]:
import pickle

with open("log.pickle", 'wb') as handle:
        pickle.dump(log, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
[
    ['Как приготовить карбонару?', 'Что нужно для соуса?'],
    ['Что приготовить детского с мясом?'],
    ['Быстрый рецепт пирога из яблок?'],
    ['Что сделать из кабачков?'],
    ['У меня есть картошка, морковь и духовка. Что мне можно сделать?'],
    ['Дай мне рецепт жареной рыбы с картофелем.'],
    ['Мои дети голодные, а у меня нет времени. Как их накормить?'],
    ['Что нужно для фруктового салата со сладкими фруктами.','Какие фрукты можно использовать вместо дыни?'],
]